In [1]:
import cv2
import numpy as np

In [203]:
cv2.__version__

'4.5.5'

In [204]:
human=cv2.imread("./data_2/human.jpg")

In [205]:
type(human)

numpy.ndarray

In [206]:
human_copy=human.copy()

In [207]:
# line 2&3 use to dont get error when we close the output window
cv2.imshow('human',human)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [208]:
human.shape

(510, 768, 3)

In [209]:
#mask
mask=np.zeros(human.shape[:2], np.uint8) # mask=np.zeros(human.shape[:-1]) that same

In [210]:
# creat dummy array for fetures using
bgd=np.zeros((1,65),np.float64) # creat background
fgd=np.zeros((1,65),np.float64) # creat front background

In [211]:
# how to select region using openv => by using  selectro function 
# thes function help me to find thw w&h for the obj which i need
rect=cv2.selectROI(human)

In [212]:
cv2.imshow('human',rect)
cv2.waitKey(10)
cv2.destroyAllWindows()

In [213]:
x,y,w,h=rect

In [214]:
cv2.rectangle(human_copy,(x,y),(x+w,y+h),(0,0,255),3)

array([[[34, 60, 67],
        [31, 57, 64],
        [29, 54, 64],
        ...,
        [30, 56, 63],
        [32, 55, 63],
        [32, 55, 63]],

       [[34, 60, 67],
        [34, 60, 67],
        [33, 58, 68],
        ...,
        [28, 54, 61],
        [32, 54, 65],
        [32, 55, 63]],

       [[36, 61, 71],
        [35, 60, 70],
        [33, 58, 68],
        ...,
        [28, 53, 63],
        [33, 55, 67],
        [34, 56, 67]],

       ...,

       [[37, 55, 38],
        [38, 56, 39],
        [35, 54, 37],
        ...,
        [54, 99, 82],
        [51, 95, 78],
        [46, 90, 73]],

       [[36, 55, 36],
        [37, 56, 37],
        [35, 55, 36],
        ...,
        [53, 98, 82],
        [50, 95, 79],
        [46, 91, 75]],

       [[35, 54, 35],
        [37, 56, 37],
        [35, 55, 36],
        ...,
        [50, 97, 81],
        [48, 93, 77],
        [43, 88, 72]]], dtype=uint8)

In [215]:
cv2.imwrite("Human_ROI.png",human_copy)

True

In [216]:
# algorithm grabcut we cut original image according the rect image 
#5==num of iteration
cv2.grabCut(human,mask,rect,bgd,fgd,5,cv2.GC_INIT_WITH_RECT)
# this fun return 3 valuse one respect bgd,one respect values we find by ROI and the last one respect human

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[2.08483387e-01, 1.28057734e-01, 3.20413790e-01, 1.94959354e-01,
         1.48085735e-01, 3.67911840e+01, 6.39328200e+01, 7.93200077e+01,
         2.89678475e+01, 5.13648104e+01, 4.55973370e+01, 4.52815737e+01,
         1.00596537e+02, 8.22208824e+01, 3.53602978e+01, 8.42195864e+01,
         6.55107936e+01, 5.41991720e+01, 9.89168191e+01, 1.03489208e+02,
         4.82584597e+01, 5.75894429e+01, 6.95623710e+01, 5.75894429e+01,
         9.49168047e+01, 1.14393149e+02, 6.95623710e+01, 1.14393149e+02,
         1.60479047e+02, 7.49395376e+01, 7.63019703e+01, 5.23079547e+01,
         7.63019703e+01, 1.64242857e+02, 5.38469914e+01, 5.23079547e+01,
         5.38469914e+01, 1.04779324e+02, 2.62277945e+01, 2.24325721e+01,
         2.37596293e+01, 2.24325721e+01, 2.8

In [217]:
mask.shape

(510, 768)

In [218]:
mask.min(),mask.max()

(0, 3)

In [219]:
#the most of valuse is go to be 1.14 ==>white so background 0 mask=1 ,rect=2 
mask.std()

1.1446581532004372

In [220]:
# creat mask2 we get 3values from grabcut 0,1,2
mask2 = np.where((mask == 2 ) | (mask == 0), 0, 1).astype("uint8")#last 0.1 is refer to white if true else black 
cv2.imwrite("mask.png", mask*80)
cv2.imwrite("mask2.png", mask2*255)

True

In [221]:
np.unique(mask)

array([0, 2, 3], dtype=uint8)

In [222]:
np.unique(mask2)

array([0, 1], dtype=uint8)

In [223]:
mask.ndim,mask2.ndim

(2, 2)

In [224]:
#inference on original data (image),mask2 return just human so we take it
human = human * mask2[:,:,np.newaxis]
cv2.imwrite("result.png", human)

True

## Videos 


## Cars

In [2]:
haar_cascae_car="./data_2/haarcascade_car.xml"

In [3]:
car_class = cv2.CascadeClassifier (haar_cascae_car)

In [4]:
# read cars video using fun in cv => vieo capture
cap= cv2.VideoCapture("./data_2/cars.avi")

In [5]:
# objec detection cars
#open video inside loop to continue run video 
count=0
while cap.isOpened():
    count+=1
    if count  % 10==0:# every 20 milliseconde        
        ret,frame=cap.read()        
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # convert each frame to gray
        cars=car_class.detectMultiScale(gray, 1.1, 3)#calssification , detect for every frame
        
        for(x,y,w,h) in cars:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)          
            cv2.putText(frame,"car",(x,y),cv2.FONT_HERSHEY_SIMPLEX ,0.5, (0,0,255))  # fun control the put text like the type of font the coordinate ,...etc // note we take this argument from document
        cv2.imshow("cars",frame)
    if cv2.waitKey(0) & 0xFF == ord("q"):  # waitKey    
        break 
cap.release()
cv2.destroyAllWindows()
            